# Analisi del Sentiment sulle Traduzioni del Dataset ASLLRP - 7 Classi

**Obiettivo dello script:**
Questo script estende l'analisi del sentiment sul dataset ASLLRP implementando un sistema di classificazione a 7 classi invece di 3. L'obiettivo è ottenere una granularità maggiore nell'analisi delle emozioni delle traduzioni in inglese delle frasi in Lingua dei Segni Americana (ASL).

### Le 7 Classi di Emozioni:

1. **Estremamente Negativo** (Strongly Negative / Extremely Negative)
2. **Negativo** (Negative)
3. **Un po' Negativo** (Somewhat Negative)
4. **Neutro** (Neutral)
5. **Un po' Positivo** (Somewhat Positive)
6. **Positivo** (Positive)
7. **Estremamente Positivo** (Strongly Positive / Extremely Positive)

**Funzionamento dello script:**

1.  **Caricamento e Pulizia dei Dati:**

    - Carica il file `utterances_with_translations.csv` generato dallo script `03_matching_videos_utterances_ASLLRP.ipynb`
    - Rimuove le righe con traduzioni non valide o mancanti

2.  **Analisi del Sentiment con VADER:**

    - Utilizza VADER per calcolare il `compound_score` per ogni traduzione
    - Il punteggio è normalizzato tra -1 (massimamente negativo) e +1 (massimamente positivo)

3.  **Classificazione in 7 Classi:**

    - Utilizza la **Configurazione 2 - Concentrata sul Neutro** (come definita nel notebook precedente)
    - Soglie conservative per le classi estreme (±0.75)
    - Area neutrale più ampia (-0.1 a 0.1) per riflettere la realtà linguistica

4.  **Visualizzazione e Salvataggio:**
    - Genera grafici per visualizzare la distribuzione delle 7 classi
    - Salva i risultati in CSV per le fasi successive del progetto


In [23]:
# Importazione delle librerie necessarie
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import Counter
import matplotlib

# Imposta il backend di Matplotlib per evitare problemi di visualizzazione
matplotlib.use("Agg")

## Fase 1: Caricamento e Pulizia dei Dati


In [24]:
# Percorso del file CSV con le traduzioni
csv_path = "../../data/processed/utterances_with_translations.csv"

# Carica il file CSV
print("Caricamento del file CSV...")
df = pd.read_csv(csv_path)

print(f"\nColonne disponibili: {list(df.columns)}")
print(f"Numero totale di righe: {len(df)}")

# Rimuovi le righe dove la traduzione non è stata trovata, è vuota o non è una stringa valida
df_cleaned = df.dropna(subset=["caption"])
df_cleaned = df_cleaned[
    ~df_cleaned["caption"].str.contains(
        "Traduzione non trovata|Translation tag vuota", na=False
    )
]

print(f"\nNumero di righe dopo la pulizia: {len(df_cleaned)}")
print(f"Righe scartate: {len(df) - len(df_cleaned)}")

Caricamento del file CSV...

Colonne disponibili: ['video_name', 'Source collection', 'caption']
Numero totale di righe: 2127

Numero di righe dopo la pulizia: 2118
Righe scartate: 9


In [25]:
# Visualizza le righe scartate (opzionale)
discarded_indices = df.index.difference(df_cleaned.index)
discarded_rows = df.loc[discarded_indices]

print("\nRighe scartate durante la pulizia:")
if not discarded_rows.empty:
    print(f"Totale: {len(discarded_rows)}")
    print(discarded_rows[["video_name", "caption"]].head())
else:
    print("Nessuna riga scartata.")


Righe scartate durante la pulizia:
Totale: 9
        video_name                 caption
30    45035462.mp4  Traduzione non trovata
31    45107545.mp4  Traduzione non trovata
32    45239274.mp4  Traduzione non trovata
33    45404906.mp4  Traduzione non trovata
1306   8396360.mp4                     NaN


## Fase 2: Calcolo dei Punteggi di Sentiment con VADER


In [26]:
# Inizializzazione dell'analizzatore VADER
analyzer = SentimentIntensityAnalyzer()

# Calcola il compound score per ogni caption
compound_scores = []
for caption in df_cleaned["caption"]:
    scores = analyzer.polarity_scores(caption)
    compound_scores.append(scores["compound"])

df_cleaned["compound_score"] = compound_scores

print(f"Calcolato sentiment per {len(df_cleaned)} traduzioni")
print(f"\nStatistiche dei compound scores:")
print(f"Media: {np.mean(compound_scores):.4f}")
print(f"Mediana: {np.median(compound_scores):.4f}")
print(f"Deviazione standard: {np.std(compound_scores):.4f}")
print(f"Min: {np.min(compound_scores):.4f}")
print(f"Max: {np.max(compound_scores):.4f}")

Calcolato sentiment per 2118 traduzioni

Statistiche dei compound scores:
Media: 0.0908
Mediana: 0.0000
Deviazione standard: 0.3436
Min: -0.9440
Max: 0.9804


## Fase 3: Definizione delle Soglie e Classificazione a 7 Classi

Utilizziamo la **Configurazione 2 - Concentrata sul Neutro**, come stabilito nell'analisi precedente per il dataset How2Sign.


In [27]:
# Configurazione 2 - Concentrata sul Neutro
thresholds = {
    "Extremely Negative": (-1.0, -0.75),
    "Negative": (-0.75, -0.4),
    "Somewhat Negative": (-0.4, -0.1),
    "Neutral": (-0.1, 0.1),
    "Somewhat Positive": (0.1, 0.4),
    "Positive": (0.4, 0.75),
    "Extremely Positive": (0.75, 1.0),
}


def classify_emotion_7_classes(compound_score):
    """Classifica un compound score in una delle 7 classi emotive."""
    for emotion, (lower, upper) in thresholds.items():
        if lower <= compound_score <= upper:
            return emotion
    return "Neutral"  # Fallback


# Applica la classificazione
df_cleaned["emotion"] = df_cleaned["compound_score"].apply(classify_emotion_7_classes)

print("\nClassificazione completata!")
print("\nPrime 5 righe con punteggi ed emozioni:")
print(df_cleaned[["video_name", "caption", "compound_score", "emotion"]].head())


Classificazione completata!

Prime 5 righe con punteggi ed emozioni:
     video_name                                            caption  \
0   2751812.mp4            There's something strange about myself.   
1  98592049.mp4                        I never lived in a house...   
2  98857459.mp4                             I was born in Florida.   
3  98881709.mp4  I lived there in a house with my parents and t...   
4  98895873.mp4  ... while I was growing up until I was six  an...   

   compound_score            emotion  
0         -0.2023  Somewhat Negative  
1          0.0000            Neutral  
2          0.0000            Neutral  
3          0.0000            Neutral  
4          0.1779  Somewhat Positive  


## Fase 4: Analisi della Distribuzione delle 7 Classi


In [28]:
# Conta le occorrenze di ogni emozione
emotion_counts = Counter(df_cleaned["emotion"])

emotions_order = [
    "Extremely Negative",
    "Negative",
    "Somewhat Negative",
    "Neutral",
    "Somewhat Positive",
    "Positive",
    "Extremely Positive",
]

print("\nDistribuzione delle emozioni:")
print("=" * 60)
for emotion in emotions_order:
    count = emotion_counts.get(emotion, 0)
    percentage = (count / len(df_cleaned)) * 100
    print(f"{emotion:25s}: {count:5d} ({percentage:5.2f}%)")

print(f"\nTotale: {len(df_cleaned)}")


Distribuzione delle emozioni:
Extremely Negative       :     9 ( 0.42%)
Negative                 :   183 ( 8.64%)
Somewhat Negative        :   202 ( 9.54%)
Neutral                  :  1007 (47.54%)
Somewhat Positive        :   214 (10.10%)
Positive                 :   435 (20.54%)
Extremely Positive       :    68 ( 3.21%)

Totale: 2118


## Fase 5: Visualizzazione della Distribuzione


In [29]:
# Crea il grafico a barre
plt.figure(figsize=(14, 7))

colors = ["#8B0000", "#DC143C", "#FFA07A", "#808080", "#90EE90", "#32CD32", "#006400"]
counts = [emotion_counts.get(emotion, 0) for emotion in emotions_order]

bars = plt.bar(range(len(emotions_order)), counts, color=colors)
plt.title(
    "Distribuzione delle Emozioni nel Dataset ASLLRP (7 Classi)",
    fontsize=14,
    fontweight="bold",
)
plt.xlabel("Emozione", fontsize=12)
plt.ylabel("Numero di Video", fontsize=12)
plt.xticks(
    range(len(emotions_order)),
    ["Estr.\nNeg.", "Neg.", "Po'\nNeg.", "Neutro", "Po'\nPos.", "Pos.", "Estr.\nPos."],
    fontsize=10,
)
plt.grid(axis="y", alpha=0.3)

# Aggiungi i numeri sopra ogni barra
for bar in bars:
    height = bar.get_height()
    if height > 0:
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{int(height)}",
            ha="center",
            va="bottom",
            fontsize=10,
        )

# Aggiungi informazioni sulla configurazione
config_text = "Config: Concentrata sul Neutro\nSoglie: ±0.75 (estremi), ±0.4, ±0.1"
plt.text(
    0.98,
    0.98,
    config_text,
    transform=plt.gca().transAxes,
    fontsize=9,
    verticalalignment="top",
    horizontalalignment="right",
    bbox=dict(facecolor="white", alpha=0.7, edgecolor="gray"),
)

plt.tight_layout()

# Salva il grafico
output_folder = "../../reports/figures"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "asllrp_emotions_distribution_7_classes.png")
plt.savefig(output_path, dpi=300, bbox_inches="tight")
plt.close()

print(f"\nGrafico salvato in: {output_path}")


Grafico salvato in: ../../reports/figures/asllrp_emotions_distribution_7_classes.png


## Fase 6: Analisi della Distribuzione dei Compound Scores


In [30]:
# Visualizzazione della distribuzione con istogramma
plt.figure(figsize=(14, 6))

# Subplot 1: Istogramma dettagliato
plt.subplot(1, 2, 1)
plt.hist(compound_scores, bins=50, color="skyblue", edgecolor="black", alpha=0.7)
plt.title(
    "Distribuzione dei Punteggi Compound (ASLLRP)", fontsize=12, fontweight="bold"
)
plt.xlabel("Valore Compound", fontsize=10)
plt.ylabel("Frequenza", fontsize=10)
plt.axvline(x=0, color="red", linestyle="--", linewidth=1, label="Zero")

# Aggiungi linee per le soglie principali
for threshold_val in [-0.75, -0.4, -0.1, 0.1, 0.4, 0.75]:
    plt.axvline(
        x=threshold_val, color="orange", linestyle=":", linewidth=0.8, alpha=0.5
    )

plt.grid(axis="y", alpha=0.3)
plt.legend()

# Subplot 2: Box plot
plt.subplot(1, 2, 2)
plt.boxplot(compound_scores, vert=True)
plt.title("Box Plot dei Punteggi Compound (ASLLRP)", fontsize=12, fontweight="bold")
plt.ylabel("Valore Compound", fontsize=10)
plt.axhline(y=0, color="red", linestyle="--", linewidth=1)
plt.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.savefig(
    os.path.join(output_folder, "asllrp_compound_distribution_7_classes.png"),
    dpi=300,
    bbox_inches="tight",
)
plt.close()

print("Grafico della distribuzione dei compound scores salvato!")

Grafico della distribuzione dei compound scores salvato!


## Fase 7: Esempi Rappresentativi per Classe


In [31]:
# Mostra esempi per ogni classe
print("\n" + "=" * 80)
print("ESEMPI RAPPRESENTATIVI PER OGNI CLASSE")
print("=" * 80)

for emotion in emotions_order:
    examples = df_cleaned[df_cleaned["emotion"] == emotion]

    if not examples.empty:
        print(f"\n{emotion.upper()}:")
        print("-" * 80)

        # Mostra fino a 3 esempi
        for i, (idx, row) in enumerate(examples.head(3).iterrows(), 1):
            print(f"\nEsempio {i}:")
            print(f"  Video: {row['video_name']}")
            print(f"  Compound Score: {row['compound_score']:.4f}")
            caption_preview = (
                row["caption"][:150] + "..."
                if len(row["caption"]) > 150
                else row["caption"]
            )
            print(f"  Caption: {caption_preview}")
    else:
        print(f"\n{emotion.upper()}: Nessun esempio trovato")


ESEMPI RAPPRESENTATIVI PER OGNI CLASSE

EXTREMELY NEGATIVE:
--------------------------------------------------------------------------------

Esempio 1:
  Video: 254728.mp4
  Compound Score: -0.8977
  Caption: It is very intimidating. There will be seven professionals informing the two parents that their child seem to be sick or have the "cancer".

Esempio 2:
  Video: 257368.mp4
  Compound Score: -0.9440
  Caption: They often use the word "diagnosis" It's very negative and it's used for more negative references, such as sickness or cancer. Or maybe terminal illne...

Esempio 3:
  Video: 1194707.mp4
  Compound Score: -0.7909
  Caption: Stop lying, it's not funny. Where's mom?

NEGATIVE:
--------------------------------------------------------------------------------

Esempio 1:
  Video: 99395897.mp4
  Compound Score: -0.4404
  Caption: so I didn`t need to worry about attending a hearing school or a Deaf school.

Esempio 2:
  Video: 25328.mp4
  Compound Score: -0.4767
  Caption: There a

## Fase 8: Salvataggio dei Risultati


In [32]:
# Prepara il DataFrame per il salvataggio
output_df = df_cleaned[["video_name", "caption", "compound_score", "emotion"]].copy()

# Percorso del file CSV di output
output_csv_path = "../../data/processed/asllrp_video_sentiment_data_7_classes.csv"

# Crea la directory se non esiste
os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

# Salva il DataFrame in CSV
output_df.to_csv(output_csv_path, index=False)

print(f"\nFile CSV salvato con successo in: {output_csv_path}")
print(f"Numero di righe salvate: {len(output_df)}")
print(f"Colonne: {list(output_df.columns)}")


File CSV salvato con successo in: ../../data/processed/asllrp_video_sentiment_data_7_classes.csv
Numero di righe salvate: 2118
Colonne: ['video_name', 'caption', 'compound_score', 'emotion']


In [33]:
# Salva anche un file di documentazione con le soglie usate
thresholds_path = "../../data/processed/asllrp_thresholds_7_classes.txt"

with open(thresholds_path, "w", encoding="utf-8") as f:
    f.write("Configurazione utilizzata: Concentrata sul Neutro\n")
    f.write(f"Data: {pd.Timestamp.now()}\n")
    f.write("Dataset: ASLLRP\n")
    f.write("\nSoglie:\n")
    f.write("=" * 60 + "\n")

    for emotion, (lower, upper) in thresholds.items():
        f.write(f"{emotion:25s}: {lower:7.4f} < compound ≤ {upper:7.4f}\n")

    f.write("\nDistribuzione:\n")
    f.write("=" * 60 + "\n")
    for emotion in emotions_order:
        count = emotion_counts.get(emotion, 0)
        percentage = (count / len(df_cleaned)) * 100
        f.write(f"{emotion:25s}: {count:5d} ({percentage:5.2f}%)\n")

    f.write(f"\nTotale righe: {len(df_cleaned)}\n")

print(f"\nDocumentazione soglie salvata in: {thresholds_path}")


Documentazione soglie salvata in: ../../data/processed/asllrp_thresholds_7_classes.txt


## Riepilogo Finale

✅ **Completato con successo!**

### File generati:

1. **CSV principale**: `asllrp_video_sentiment_data_7_classes.csv` - Contiene tutte le traduzioni classificate
2. **Documentazione**: `asllrp_thresholds_7_classes.txt` - Documenta le soglie e la distribuzione
3. **Grafici**:
   - Distribuzione delle 7 classi
   - Analisi dei compound scores

### Prossimi passi:

- Utilizzare il notebook `03_find_golden_labels_7_classes.ipynb` per identificare le golden labels
- Creare il test set basato sulle corrispondenze con EmoSign
- Procedere con l'addestramento dei modelli usando le 7 classi
